In [27]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/nlp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/nlp'
/content/drive/MyDrive/nlp


In [28]:
import pandas as pd
import numpy as np
import pickle
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [29]:
# Load the dataset into a Pandas dataframe
train_df = pd.read_csv("training_data.csv")

In [31]:
# Preprocess the text data
nltk.download("stopwords")
stop_words = stopwords.words("english")
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = [word for word in text.split() if word not in stop_words]
    return " ".join(text)
train_df["complaint_text"] = train_df["complaint_text"].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
# Split the dataset into training and testing sets
X = train_df["complaint_text"].values
y = train_df["Topic"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Use TfidfVectorizer to convert the preprocessed text data into feature vectors
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [34]:
# Train a Support Vector Machine (SVM) classifier on the training set
classifier = SVC(kernel="linear")
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [35]:
# Evaluate the model on the testing set
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[1066   19   35    8   34]
 [  34  775   19    8   34]
 [  28    7  913    9   13]
 [  18   16   13  400    3]
 [  40   21   22    8  672]]
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1162
           1       0.92      0.89      0.91       870
           2       0.91      0.94      0.93       970
           3       0.92      0.89      0.91       450
           4       0.89      0.88      0.88       763

    accuracy                           0.91      4215
   macro avg       0.91      0.90      0.91      4215
weighted avg       0.91      0.91      0.91      4215



In [ ]:
with open("../models/svc_model.pcl") as save_file:
    pickle.dump(classifier, save_file)